# Day 1 Exercises

In [12]:
from biocrnpyler import *
import numpy as np
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()
import colorcet

Loading BokehJS ...

In [13]:
palette = colorcet.b_glasbey_category10

In [14]:
# 2.a
# create species
G = Species("G", material_type="promoter")
R = Species("R", material_type="repressor")
T = Species("T", material_type="transcript")
# create complex
GnR = Complex([G,R,R,R])
# rates
kb = 1
ku = 1
kx = 5
# create reactions
repressor_R = Reaction.from_massaction([G]+3*[R], [GnR], k_forward=kb, k_reverse=ku)
transcription_R = Reaction.from_massaction([G], [G, T], k_forward=kx)
CRN_2a = ChemicalReactionNetwork(species = [G,R,T], reactions = [repressor_R, transcription_R])
print(CRN_2a.pretty_print())



Species (4) = {0. promoter[G] init_conc = 0, 1. repressor[R] init_conc = 0, 2. transcript[T] init_conc = 0, 3. complex[promoter[G]:3x_repressor[R]] init_conc = 0}

Reactions (2) = [
0. promoter[G]+3repressor[R] <--> complex[promoter[G]:3x_repressor[R]]
 Kf=k_forward * promoter_G * repressor_R^3
 Kr=k_reverse * complex_promoter_G_repressor_R_3x_
  k_forward=1
  k_reverse=1

1. promoter[G] --> promoter[G]+transcript[T]
 Kf=k_forward * promoter_G
  k_forward=5

]


In [20]:
# create initial condition dictionary
x0_2 = {str(G):100,
     str(T):100,
       str(R):100}
timepoints = np.linspace(0,100,500)
results_2a = CRN_2a.simulate_with_bioscrape(timepoints, x0_2)

p_a2 = bokeh.plotting.figure(width = 450, height = 350, title= 'Exercise 2 a',
                           x_axis_label = 'time',
                           y_axis_label = 'concentration')
p_a2.line(timepoints, results_2a['promoter_G'], legend_label = 'G', color = palette[0], line_width=2)
p_a2.line(timepoints, results_2a['transcript_T'], legend_label = 'T', color = palette[1], line_width=2)
p_a2.line(timepoints, results_2a['repressor_R'], legend_label = 'R', color = palette[2], line_width=2)
p_a2.line(timepoints, results_2a['complex_promoter_G_repressor_R_3x_'], legend_label = 'GnR', color = palette[3], line_width=2) 

bokeh.io.show(p_a2)

In [17]:
results_2a

,promoter_G,repressor_R,transcript_T,complex_promoter_G_repressor_R_3x_,time
0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.200401
2,0.0,0.0,0.0,0.0,0.400802
3,0.0,0.0,0.0,0.0,0.601202
4,0.0,0.0,0.0,0.0,0.801603
...,...,...,...,...,...
495,0.0,0.0,0.0,0.0,99.198397
496,0.0,0.0,0.0,0.0,99.398798
497,0.0,0.0,0.0,0.0,99.599198
498,0.0,0.0,0.0,0.0,99.799599


# Exercise 1:
### A. Create a CRN to simulate the following enzymatic reaction using massaction propensities: $A + C \leftrightarrow A:C \rightarrow C + B$
(Note the first reaction is reversible and you can choose any rates $k_b$ and $k_u$. The second reaction isnt reversible and occurs at rate $k_{cat}$)



In [15]:
# Create Species
A = Species('A')
B = Species('B')
C = Species('C')

# Create Complex
Complex_AC = Complex([A, C], ordered = True)

# Rate
kb = 1
ku = 1
kcat = 2

# Create Reactions
R1_1a = Reaction.from_massaction([A, C], [Complex_AC], k_forward = kb, k_reverse = ku)

R2_1a = Reaction.from_massaction([Complex_AC], [C, B], k_forward = kcat)

# Create CRN
CRN_1a = ChemicalReactionNetwork(species = [A, B, C, Complex_AC], reactions = [R1_1a, R2_1a])

In [16]:
print(CRN_1a.pretty_print(show_rates = True))

Species (4) = {0. A init_conc = 0, 1. B init_conc = 0, 2. C init_conc = 0, 3. ordered_complex[A:C] init_conc = 0}

Reactions (2) = [
0. A+C <--> ordered_complex[A:C]
 Kf=k_forward * A * C
 Kr=k_reverse * ordered_complex_A_C_
  k_forward=1
  k_reverse=1

1. ordered_complex[A:C] --> C+B
 Kf=k_forward * ordered_complex_A_C_
  k_forward=2

]


### B. Create a different CRN to simulate the following enzymatic reaction using a ProportionalHillPositive Propensity: $A \rightarrow B \quad \rho = k_{cat} C \frac{\frac{A}{K}}{1+\frac{A}{K}}$ where $K = \frac{k_u}{k_b}$



In [18]:
# Create Species
A = Species('A')
B = Species('B')
C = Species('C')

# Create Rate Values
kb = 1
ku = 1
kcat = 2

# Create positive hill propensity
prop_hill_pos = ProportionalHillPositive(k = kcat , s1 = A, K = ku/kb, n = 1, d = C)

# Create Reaction
R_prop_hill_pos = Reaction([A], [B], propensity_type = prop_hill_pos)

# Create CRN
CRN_1b = ChemicalReactionNetwork(species = [A, B, C], reactions = [R_prop_hill_pos])

In [20]:
print(CRN_1b.pretty_print())

Species (3) = {0. A init_conc = 0, 1. B init_conc = 0, 2. C init_conc = 0}

Reactions (1) = [
0. A --> B
 Kf = k C A^n/(1+(A/K)^n)
  k=2
  K=1.0
  n=1

]


### C. for what rates $k_u$, $k_b$, and $k_{cat}$ do the two models agree and disagree?

In [24]:
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import numpy as np

from bokeh.layouts import row

import colorcet
palette = colorcet.b_glasbey_category10

Loading BokehJS ...

Simulate

In [29]:
# Creating initial condition dict
x0 = {str("A"):1,
     str("C"):1}
timepoints = np.linspace(0,5,50)

Results_1a = CRN_1a.simulate_with_bioscrape(timepoints, x0)
Results_1b = CRN_1b.simulate_with_bioscrape(timepoints, x0)

In [30]:
p_a = bokeh.plotting.figure(width = 450, height = 350, title = 'Exercise 1a', 
                         x_axis_label = 'time',
                         y_axis_label = 'conc')
p_a.line(timepoints, Results_1a['A'], legend_label = 'A', color = palette[0], line_width = 2)
p_a.line(timepoints, Results_1a['B'], legend_label = 'B',color = palette[1], line_width = 2)
p_a.line(timepoints, Results_1a['C'], legend_label = 'C',color = palette[2], line_width = 2)
p_a.line(timepoints, Results_1a['ordered_complex_A_C_'], legend_label = 'AC',color = palette[3], line_width = 2)


p_b = bokeh.plotting.figure(width = 450, height = 350, title = 'Exercise 1b', 
                         x_axis_label = 'time',
                         y_axis_label = 'conc')
p_b.line(timepoints, Results_1b['A'], legend_label = 'A', color = palette[0], line_width = 2)
p_b.line(timepoints, Results_1b['B'], legend_label = 'B',color = palette[1], line_width = 2)
p_b.line(timepoints, Results_1b['C'], legend_label = 'C',color = palette[2], line_width = 2)

bokeh.io.show(row(p_a, p_b))